In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import warnings
from nltk.corpus import stopwords
warnings.filterwarnings("ignore",category=DeprecationWarning)
%matplotlib inline

#https://na80.lightning.force.com/lightning/r/Report/00O1P000003mYjbUAE/view

/Users/emeryames/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('year_data.csv')
df.Description.head()

0    Chat transcript\n\n\n\t\n\t\t\n\t\t\tName: GAr...
1    Chat transcript\n\n\n\t\n\t\t\n\t\t\tName: Gen...
2    Chat transcript\n\n\n\t\n\t\t\n\t\t\tE-mail: c...
3    Chat transcript\n\n\n\t\n\t\t\n\t\t\tE-mail: c...
4    Chat transcript\n\n\n\t\n\t\t\n\t\t\tE-mail: e...
Name: Description, dtype: object

In [3]:
data = df.Description.values.tolist()
data = [re.sub('\S*@\S*\s?', "", sent) for sent in data]
data = [re.sub('\s+', " ", sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub('https:\S+',"", sent) for sent in data]
data = [re.sub('Chat transcript',"", sent) for sent in data]
data = [re.sub('BevSpot Support here! How can we help',"", sent) for sent in data]
data = [re.sub('Chat has been opened from',"", sent) for sent in data]
data = [re.sub('E-mail', "", sent) for sent in data]
data = [re.sub('..:..:.. ..', "", sent) for sent in data]
data = [re.sub('America\S+', "", sent) for sent in data]
data = [re.sub('\S+.com\S+', "", sent) for sent in data]
data = [re.sub('BevSpot', "", sent) for sent in data]

pprint(data[:1])

[' Name: GAry Hipko : Brad Mon, 01/04/21   Hi from  Support! Check out our '
 'help center at  or chat us here if you have any questions as you move around '
 'the software. GAry Hipko  my food account is locked. why? Brad  Hi, Gary let '
 'me look into this for you. Gary, I am seeing a delinquency on the food '
 'account. Your grace period ended on 12/10. I am going to see if I can talk '
 'to your account manager. Do you mind waiting a few minutes? GAry Hipko  ive '
 'been waiting 1 1/2 hours for a call back. whats a few more minutes... Brad  '
 'Gary, I am very sorry to hear that. Did you have an appointment already '
 'scheduled? GAry Hipko  no i left a message Brad  Okay. Let me see what I can '
 'do. Gary, are you still with me? GAry Hipko  yes Brad  I am going to send '
 'you a link to Susans calendar so you can schedule an appointment to speak '
 'with her. I am also going to temporarily unlock the account until you '
 'resolve the billing issue with her. GAry Hipko  thanks Bra

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data))

In [5]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [6]:
stop_words = stopwords.words('english')
stop_words.extend (('chat','let','go','see','thank','get','done','email','take','look','hang','tight','show','reach','send','thanks',
                    'im','one','help','went','said','hi','gm','way','best','need','one','got','much','want','check','youll','sure',
                    'hello','hear','us','name','ok','okay','archived','yes','no','well','like','would','link','list','ill',
                    'mon','tue','wed','thu','fri',
                    'erick','victoria','marguerite','dan'))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

#def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#    texts_out = []
#    for sent in texts:
#        doc = nlp(" ".join(sent)) 
#        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#    return texts_out

In [8]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)
#names get clustered with corresponding last name. can filter out for better model
#fuck. it messes with cart_builder, last_week, negative_usage
#data_words_bigrams = str(data_words_bigrams)
#data_words_bigrams = re.sub('\S+_\S+', "", data_words_bigrams)
#data_words_trigrams = make_trigrams(data_words_nostops)

#manually choose bigram or trigram
data = data_words_bigrams

In [ ]:
print(data_words_bigram)

In [9]:
id2word = corpora.Dictionary(data)
texts = data
corpus = [id2word.doc2bow(text) for text in texts]

In [10]:
num_topics = 14
chunksize = 200 
passes = 25 
iterations = 1000
eval_every = 1 

lda_model = LdaModel(corpus=corpus,
    id2word=id2word,
    num_topics = num_topics, 
    random_state = 100,
    update_every = eval_every,
    chunksize=chunksize,
    eta = 'auto',
    passes = passes,
    iterations = iterations,
    minimum_phi_value = .95,
    alpha = 0.01,
    per_word_topics = True)
#score
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.232345319606211

Coherence Score:  0.3779369505143094


In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, alpha = 0.01, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data, dictionary=id2word, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data, start=2, limit=20, step=1)

# Elbow for best num_topics. 100% fucked until i figure it out
import matplotlib.pyplot as plt
limit=20; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.show()

In [11]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.223414  0.190402       1        1  30.068632
0     -0.070923  0.189552       2        1  17.888679
2     -0.024085  0.380811       3        1   8.889693
8     -0.329643  0.081623       4        1   8.114708
5      0.144553  0.243575       5        1   7.925197
7      0.298043  0.120401       6        1   4.860886
3     -0.283186 -0.157153       7        1   3.821660
1      0.015433 -0.303459       8        1   3.432071
9      0.201006  0.001951       9        1   3.229778
4     -0.063265 -0.068542      10        1   2.907168
13    -0.127688 -0.246550      11        1   2.774258
10     0.263217 -0.169370      12        1   2.739678
12     0.135146 -0.177421      13        1   2.662623
6      0.064805 -0.085821      14        1   0.684968, topic_info=               Term        Freq       Total Category  logprob  loglift
1380         kendra  152.000000  152.000000  Default  30.0000  30.0000
394         invoice  116.000000  116.000000  Default  29.0000  29.0000
2           account  296.000000  296.000000  Default  28.0000  28.0000
47        inventory  187.000000  187.000000  Default  27.0000  27.0000
431           sales  121.000000  121.000000  Default  26.0000  26.0000
...             ...         ...         ...      ...      ...      ...
836        seperate    0.290854    2.449404  Topic14  -6.0474   2.8528
991            ways    0.290854    2.317508  Topic14  -6.0474   2.9081
1005            saw    0.290854    8.800651  Topic14  -6.0474   1.5738
1152  profitability    0.290854    1.988331  Topic14  -6.0474   3.0613
1217           full    0.290854    1.956794  Topic14  -6.0474   3.0773

[748 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.746350     able
0         2  0.022617     able
0         4  0.056542     able
0         5  0.124392     able
0         6  0.011308     able
...     ...       ...      ...
113      11  0.141402    youre
864      10  0.950126  zachary
1537      2  0.280567     zero
1537     10  0.140283     zero
1537     13  0.561133     zero

[1254 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 1, 3, 9, 6, 8, 4, 2, 10, 5, 14, 11, 13, 7])